## Start with a markov model 

In [106]:
import numpy as np

# position 0 is always an absorbing state

# a transition matrix would satisfy, each row sums to 1

# for convenience, N will indicate N potential products, but the actual matrix will be (n+1)*(n+1) due to potision 0
def gen_transition(N):
    base = np.identity(N+1)
    
    sub_M = np.random.uniform(low=0., high=1, size=(N, N))
    
    sub_M[sub_M < 1 - 1/np.sqrt(N)]=0   # to maintain a sparse structure
    
    base[1:N+1,1:N+1] = sub_M
    
    base[1:N+1,0] = np.random.uniform(low=0.1, high=0.3, size = N)
    
    base = base / base.sum(axis=1, keepdims = True)
    
    return base

# share the probability on diagonal to other places
def gen_transition_nodiag(N):
    mat = gen_transition(N)
    for i in range(1,N+1):
        mat[i,i]=0
        
    mat = mat / mat.sum(axis=1, keepdims = True)
    return mat

# utilitiy, from probability vector to index_indicator
def indicator(probs):
    if not sum(probs) > 1.0 - 1e-6 and sum(probs) < 1.0 + 1e-6:
        print("WRONG PROBABILITY!")
        return
    index = np.random.choice(len(probs), 1, p=probs)[0]
    indicate = np.zeros(len(probs))
    indicate[index] = 1
    return indicate


# transition matrix should be given in advance, for sure!
# given an assortment, generate a hitting sample
def single_sample_generator(Lams, TransP, Assorts):
    Lams = np.insert(Lams, 0, 0)
    Assorts = np.insert(Assorts, 0, 0)
    choice = indicator(Lams)
    while True:
        if choice[0] == 1 or np.dot(choice, Assorts)>0:
            return choice
        else:
            choice_prob = TransP[np.squeeze(np.argwhere(choice == 1))]
            choice = indicator(choice_prob)

def multi_sample_generator(Lams, TransP, Assorts, sampleN):
    N = len(Lams)
    choices = np.zeros((sampleN,N+1))
    for i in range(sampleN):
        choices[i] = single_sample_generator(Lams, TransP, Assorts)
        
    return choices
    
# the exact probability for product choice    
def actual_prob(Lams, TransP, Assorts):
    Lams = np.insert(Lams, 0, 0)
    Assorts = np.insert(Assorts, 0, 1)
    S_plus = np.squeeze(np.argwhere(Assorts == 1),axis=1)
    S_bar = np.squeeze(np.argwhere(Assorts == 0),axis=1)
    B = TransP[np.expand_dims(S_bar, axis=1), S_plus]
    C = TransP[np.expand_dims(S_bar, axis=1), S_bar]
    
    distri = np.zeros(len(Lams)+1)
    
    addi = np.matmul(np.matmul(np.expand_dims(Lams[S_bar], axis=0), np.linalg.inv(np.identity(len(C)) - C)), B)
    
    count = 0
    for i in S_plus:
        distri[i] = Lams[i] + addi[0,count]
        count += 1

    return distri
    
    

    
transP = gen_transition_nodiag(4)
lams = np.array([0.1,0.4,0.3,0.2])
assortment = np.array([0,1,1,0])
# bundle = np.squeeze(np.argwhere(assortment == 1))
actual_prob(lams, transP, assortment)

array([0.25409705, 0.        , 0.4       , 0.34590295, 0.        ,
       0.        ])

##  We use multi_sample_generator to test actual_prob

In [107]:
transP = gen_transition_nodiag(10)
lams = np.array([0.1]*10)
assortment = np.zeros(10)
assortment[[1,3,5,6,8]]=1
print(assortment)

# bundle = np.squeeze(np.argwhere(assortment == 1))
print(actual_prob(lams, transP, assortment))

data = np.sum(multi_sample_generator(lams, transP, assortment, 10000), axis=0)
data

[0. 1. 0. 1. 0. 1. 1. 0. 1. 0.]
[0.05880171 0.         0.13522966 0.         0.20684795 0.
 0.2202711  0.18126796 0.         0.19758162 0.         0.        ]


array([ 613.,    0., 1338.,    0., 2100.,    0., 2205., 1807.,    0.,
       1937.,    0.])

## After the checks above, we start building the network to check its representation power

### start with some experiments

In [1]:
import torch
import torch.nn as nn

OSError: dlopen(/Users/turquoisekitty/opt/anaconda3/envs/working/lib/python3.8/site-packages/torch/lib/libtorch_global_deps.dylib, 10): Library not loaded: @rpath/libmkl_intel_thread.dylib
  Referenced from: /Users/turquoisekitty/opt/anaconda3/envs/working/lib/python3.8/site-packages/torch/lib/libtorch_global_deps.dylib
  Reason: image not found